In [1]:
#!/usr/bin/env python
# coding: utf-8

In [8]:
import pandas as pd
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from tqdm import tqdm

# record start time
_START_RUNTIME = time.time()

# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# Define data and weight path
DATA_PATH = "../data/raw/fitzpatrick17k.csv"

## About Raw Data
https://github.com/mattgroh/fitzpatrick17k/blob/main/fitzpatrick17k.csv

In [ ]:
# Analysis of the Fitzpatrick 17k dataset

df = pd.read_csv(DATA_PATH)

print("Columns in the dataset:", df.columns)
print("Total samples:", len(df))
print("\nFitzpatrick scale distribution:")
print(df['fitzpatrick_scale'].value_counts().sort_index())
print("\nNumber of unique conditions:", df['label'].nunique())
print("\nTop 10 conditions:")
print(df['label'].value_counts().head(10))
print(['label']).unique()

Columns in the dataset: Index(['md5hash', 'fitzpatrick_scale', 'fitzpatrick_centaur', 'label',
       'nine_partition_label', 'three_partition_label', 'qc', 'url',
       'url_alphanum'],
      dtype='object')
Total samples: 16577

Fitzpatrick scale distribution:
fitzpatrick_scale
-1     565
 1    2947
 2    4808
 3    3308
 4    2781
 5    1533
 6     635
Name: count, dtype: int64

Number of unique conditions: 114

Top 10 conditions:
label
psoriasis                      653
squamous cell carcinoma        581
lichen planus                  491
basal cell carcinoma           468
allergic contact dermatitis    430
lupus erythematosus            410
neutrophilic dermatoses        361
sarcoidosis                    349
photodermatoses                348
folliculitis                   342
Name: count, dtype: int64
['label']


AttributeError: 'NoneType' object has no attribute 'unique'

# Process the data and organize data into right folders